In [ ]:
import sys
import os
import tensorflow.keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import preprocess_input
import pandas as pd
from sklearn.model_selection import train_test_split
import PIL.Image as Image
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=sys.maxsize)

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
five_labels_df = pd.read_csv('/content/drive/My Drive/BachelorProject/dataframe/five_labels_df.csv', index_col=0)

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold

Y = five_labels_df[['Label']]

skf = StratifiedKFold(n_splits = 2, random_state = 42, shuffle = True) 

In [ ]:
from keras_preprocessing.image import ImageDataGenerator
import tensorflow as tf


datagen=ImageDataGenerator(rescale=1./255, preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

In [ ]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [ ]:
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

def createModel():
  counter = 0
  resnet50 = tf.keras.applications.resnet50
  conv_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
  for layer in conv_model.layers:
      counter += 1
      layer.trainable = False

  x = tf.keras.layers.GlobalAveragePooling2D()(conv_model.output)
  predictions = tf.keras.layers.Dense(5, activation='softmax')(x)
  full_model = tf.keras.models.Model(inputs=conv_model.input, outputs=predictions)

  return full_model

In [ ]:
from sklearn.model_selection import train_test_split

train_val, test_set = train_test_split(five_labels_df, test_size=0.2, stratify=five_labels_df.Label)

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold

Y = train_val[['Label']]

skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True) 

In [ ]:
  # CREATE NEW MODEL
  model = createModel()
  # COMPILE NEW MODEL
  model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
          metrics=['accuracy', tf.keras.metrics.Precision()])


In [ ]:
VALIDATION_ACCURACY = []
VAL_PRECISION = []
VALIDATION_LOSS = []

save_dir = '/content/drive/My Drive/BachelorProject/temp_models/'
fold_var = 1

for train_index, val_index in skf.split(np.zeros(len(Y)),Y):
  print(fold_var)
  training_data = train_val.iloc[train_index]
  validation_data = train_val.iloc[val_index]
  
  train_data_generator=datagen.flow_from_dataframe(dataframe=training_data, 
                                            x_col="FilePath", 
                                            y_col="Label", 
                                            class_mode="categorical", 
                                            target_size=(224,224), 
                                            color_mode='rgb', 
                                            save_format='jpg', 
                                            shuffle = True,
                                            batch_size=32)
  
  valid_data_generator=datagen.flow_from_dataframe(dataframe=validation_data, 
                                            x_col="FilePath", 
                                            y_col="Label", 
                                            class_mode="categorical", 
                                            target_size=(224,224), 
                                            color_mode='rgb', 
                                            save_format='jpg', 
                                            shuffle = True,
                                            batch_size=32)


  # CREATE NEW MODEL
  model = createModel()
  # COMPILE NEW MODEL
  model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
          metrics=['accuracy', tf.keras.metrics.Precision()])

  # CREATE CALLBACKS
  checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
              monitor='val_accuracy', verbose=1, 
              save_best_only=True, mode='max')
  callbacks_list = [checkpoint]
  # There can be other callbacks, but just showing one because it involves the model name
  # This saves the best model
  # FIT THE MODEL
  STEP_SIZE_TRAIN=train_data_generator.n//train_data_generator.batch_size
  STEP_SIZE_VALIDATION=valid_data_generator.n//valid_data_generator.batch_size

  history = model.fit(train_data_generator,
          epochs=5,
          steps_per_epoch=STEP_SIZE_TRAIN,
          callbacks=callbacks_list,
          validation_data=valid_data_generator,
          validation_steps=STEP_SIZE_VALIDATION)
  #PLOT HISTORY
  #		:
  #		:

  # LOAD BEST MODEL to evaluate the performance of the model
  model.load_weights("/content/drive/My Drive/BachelorProject/temp_models/model_"+str(fold_var)+".h5")

  results = model.evaluate(valid_data_generator)
  results = dict(zip(model.metrics_names,results))

  VALIDATION_ACCURACY.append(results['accuracy'])
  VALIDATION_LOSS.append(results['loss'])
  tf.keras.backend.clear_session()

  fold_var += 1

In [ ]:
from tensorflow import keras

model = keras.models.load_model("/content/drive/My Drive/BachelorProject/temp_models/model_4.h5")

In [ ]:
resnet = createModel()

94773248/94765736 [==============================] - 2s 0us/step


In [ ]:
from keras_preprocessing.image import ImageDataGenerator
import tensorflow as tf

datagen=ImageDataGenerator(rescale=1./255, preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

In [ ]:
thisdict = {
  "Bedroom": 0,
  "Kitchen": 1,
  "Living room": 2,
  "Restaurant,Bar" : 3,
  "Sport fields" : 4
}

In [ ]:
test_genarator=datagen.flow_from_dataframe(dataframe=test_set, x_col="FilePath", y_col="Label", class_mode="categorical", seed=42, 
                                            target_size=(224,224), color_mode='rgb', save_format='jpg', batch_size=10, shuffle=False)

Found 545 validated image filenames belonging to 5 classes.


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

y_pred = model.predict(test_genarator)

predicted_labels = np.argmax(y_pred, axis=-1)

real_labels = [thisdict[elem] for elem in test_set.Label]

In [ ]:
from tensorflow.keras.preprocessing import image
imgfiles = glob('/content/drive/My Drive/BachelorProject/HeatImages/living.jpg')

x = np.expand_dims(image.img_to_array(image.load_img(imgfiles[0],target_size=(224, 224, 3))),axis = 0)
x = preprocess_input(x)

In [ ]:
accuracy_score(real_labels, predicted_labels)

0.7908256880733945

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

multilabel_confusion_matrix(real_labels, predicted_labels)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
con=confusion_matrix(real_labels,predicted_labels)
levels = ['Bedroom', 'Kitchen', 'Living room', 'Restaurant,Bar', 'Sport fields']
sn.heatmap(con, xticklabels=levels, yticklabels=levels, annot=True, cmap='Greens')


In [ ]:
thisdictInvert = {
  0: "Bedroom",
  1: "Kitchen",
  2: "Living room",
  3: "Restaurant,Bar",
  4: "Sport fields"
}

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(real_labels, predicted_labels, target_names=['Bedroom', 'Kitchen', 'Living room', 'Restaurant,Bar', 'Sport fields']))